#6.1.2 CNN으로 손글씨 숫자 분류

In [52]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint

##MNIST 데이터 불러오기

In [53]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


##데이터 구조 변경

In [54]:
import numpy as np
x_train = np.reshape(x_train, (60000,28,28,1))
x_test = np.reshape(x_test, (10000,28,28,1))

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


##데이터 정규화

In [55]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_test /= gray_scale

In [56]:
# 학습 시, y(레이블)와 cross entropy를 측정해야 하므로, y를 one hot encoding으로 변환.
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# one hot encoding 변환 확인
print("첫 번째 학습 데이터 (5) : ",y_train[0])
print("첫 번째 테스트 데이터 (7) : ",y_test[0])

첫 번째 학습 데이터 (5) :  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
첫 번째 테스트 데이터 (7) :  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


##모델 생성

In [57]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5),activation='relu', input_shape=(28,28,1),padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [58]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 28, 28, 16)        416       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 14, 14, 16)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 14, 14, 32)        12832     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 1568)              0         
                                                                 
 dense_8 (Dense)             (None, 128)              

In [59]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

##조기 종료 (Early Stopping)
매 주기(Epoch)마다 검증 데이터로 검증 정확도를 측정합니다.

검증 정확도가 2번 연속으로 개선되지 않을 시, 조기 종료를 수행합니다.

In [60]:
callbacks = [EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=False),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True)]

In [61]:
model.fit(x_train, y_train, epochs=5, batch_size=500, verbose=1,validation_split = 0.1, callbacks=callbacks)

Epoch 1/5
108/108 [==============================] - 48s 436ms/step - loss: 0.4876 - accuracy: 0.8671 - val_loss: 0.1287 - val_accuracy: 0.9648
Epoch 2/5
108/108 [==============================] - 47s 434ms/step - loss: 0.1134 - accuracy: 0.9669 - val_loss: 0.0717 - val_accuracy: 0.9812
Epoch 3/5
108/108 [==============================] - 47s 434ms/step - loss: 0.0721 - accuracy: 0.9781 - val_loss: 0.0709 - val_accuracy: 0.9802
Epoch 4/5
108/108 [==============================] - 47s 432ms/step - loss: 0.0567 - accuracy: 0.9824 - val_loss: 0.0483 - val_accuracy: 0.9860
Epoch 5/5
108/108 [==============================] - 47s 433ms/step - loss: 0.0479 - accuracy: 0.9847 - val_loss: 0.0482 - val_accuracy: 0.9867


##학습 정확도

In [62]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.04400330036878586
Test accuracy: 0.9850000143051147
